In [1]:
!pip install pytorch-fid
from pytorch_fid.fid_score import calculate_fid_given_paths

  Created wheel for pytorch-fid: filename=pytorch_fid-0.2.1-py3-none-any.whl size=14835 sha256=9bc47cce67be8b20665a874aa1c762c24d53b61cb6a28f4f694fa789e7c987b6
  Stored in directory: /root/.cache/pip/wheels/24/ac/03/c5634775c8a64f702343ef5923278f8d3bb8c651debc4a6890
Successfully built pytorch-fid


In [2]:
import os
import zipfile
from zipfile import ZipFile
import requests
import tqdm
import zipfile
import requests
import numpy as np
from PIL import Image
import IPython.display
from io import BytesIO
from PIL import ImageDraw
from PIL import ImageFont
from shutil import copyfile
import six.moves.cPickle as cPickle
from typing import Optional, Tuple, List

import tensorflow as tf
import tensorflow_datasets as tfds
from keras.layers import Normalization
from keras.preprocessing.image import save_img
from keras.preprocessing.image import img_to_array

import pandas as pd
import natsort
from numpy import cov
from numpy import trace
from random import randint
from numpy import iscomplexobj
from scipy.linalg import sqrtm
from keras.applications.inception_v3 import preprocess_input

In [3]:
def download_file_from_google_drive(id, destination):
    URL = "https://docs.google.com/uc?export=download"
    session = requests.Session()
    response = session.get(URL, params = { 'id' : id }, stream = True)
    token = get_confirm_token(response)
    if token:
        params = {'id': id, 'confirm': token}
        response = session.get(URL, params = params, stream = True)
    save_response_content(response, destination)
    unpack_and_remove(destination)

def get_confirm_token(response):
    for key, value in response.cookies.items():
        if key.startswith('download_warning'):
            return value
    return None

def save_response_content(response, destination):
    chunk_size = 32768
    with open(destination, "wb") as f:
        for chunk in response.iter_content(chunk_size):
            if chunk:
                f.write(chunk)
    
def unpack_and_remove(destination):
    file_content = zipfile.ZipFile(destination)
    file_content.extractall()
    os.remove(destination)
    
file_id = '1GiktHkovd9Ii2nHNfE4P4Xkxu4GUkNRI'
destination = '/content/tensorflow-inception_images.zip'
download_file_from_google_drive(file_id, destination)

In [4]:
num_layers = 14
label_size = 2
resolution = 256

In [5]:
path = './celab_a_age/'
data_path = path + 'examples_1.tfrecord'
bucket_path = 'https://storage.googleapis.com/explaining-in-style/celeb_a_age/'
original_images = "./original_images"
data_folder = "../input/flickrfaceshq-dataset-ffhq"

In [6]:
def get_path_from_bucket(bucket_url, path):
    r = requests.get(bucket_url)
    filename = os.path.split(bucket_url)[-1].replace('.zip', '')
    zip_ref = zipfile.ZipFile(BytesIO(r.content))
    zip_ref.extractall(path)
    return os.path.join(path, filename)

generator = tf.keras.models.load_model(get_path_from_bucket(bucket_path + 'generator.savedmodel.zip', path))
encoder = tf.keras.models.load_model(get_path_from_bucket(bucket_path + 'encoder.savedmodel.zip', path))
discriminator = tf.keras.models.load_model(get_path_from_bucket(bucket_path + 'discriminator.savedmodel.zip', path))
classifier = tf.keras.models.load_model(get_path_from_bucket(bucket_path + 'mobilenet.savedmodel.zip', path))
inception = tf.keras.applications.inception_v3.InceptionV3(include_top=False, input_shape=(299, 299, 3), pooling='avg')

87924736/87910968 [==============================] - 1s 0us/step


In [7]:
def filter_unstable_images(style_change_effect: np.ndarray,
                           effect_threshold: float = 0.3,
                           num_indices_threshold: int = 750):

    unstable_images = (np.sum(np.abs(style_change_effect) > effect_threshold, axis=(1, 2, 3)) > num_indices_threshold)
    style_change_effect[unstable_images] = 0
    return style_change_effect

In [11]:
examples_url = bucket_path + 'examples_1.tfrecord'
r = requests.get(examples_url)
data_path = path + 'examples_1.tfrecord'
open(data_path, 'wb').write(r.content)
num_classes = 2
print(f'Loaded dataset: {data_path}')
table = tf.data.TFRecordDataset([data_path])
# Read sspace tfrecord unwrapped:
style_change_effect = []
latents = []
base_probs = []

for raw_record in table:
    example = tf.train.Example()
    example.ParseFromString(raw_record.numpy())
    latents.append(np.array(example.features.feature['dlatent'].float_list.value))
    seffect = np.array(example.features.feature['result'].float_list.value).reshape((-1, 2, num_classes))
    style_change_effect.append(seffect.transpose([1, 0, 2]))
    base_probs.append(np.array(example.features.feature['base_prob'].float_list.value))

style_change_effect = np.array(style_change_effect)
latents = np.array(latents)
W_values, style_change_effect, base_probs = latents, style_change_effect, np.array(base_probs)

style_change_effect = filter_unstable_images(style_change_effect, effect_threshold=2)

all_style_vectors = tf.concat(generator.style_vector_calculator(W_values, training=False)[0], axis=1).numpy()
style_min = np.min(all_style_vectors, axis=0)
style_max = np.max(all_style_vectors, axis=0)

all_style_vectors_distances = np.zeros((all_style_vectors.shape[0], all_style_vectors.shape[1], 2))
all_style_vectors_distances[:,:, 0] = all_style_vectors - np.tile(style_min, (all_style_vectors.shape[0], 1))
all_style_vectors_distances[:,:, 1] = np.tile(style_max, (all_style_vectors.shape[0], 1)) - all_style_vectors

Loaded dataset: ./celab_a_age/examples_1.tfrecord


In [12]:
def show_image(image, fmt='png'):
    if image.dtype == np.float32:
        image = np.uint8(image * 127.5 + 127.5)
    if image.shape[0] == 3:
        image = np.transpose(image, (1, 2, 0))
    bytes_io = BytesIO()
    Image.fromarray(image).save(bytes_io, fmt)
    IPython.display.display(IPython.display.Image(data=bytes_io.getvalue()))

In [13]:
all_labels = np.argmax(base_probs, axis=1)
style_effect_classes = {}
W_classes = {}
style_vectors_distances_classes = {}
all_style_vectors_classes = {}
for img_ind in range(label_size):
    img_inx = np.array([i for i in range(all_labels.shape[0]) if all_labels[i] == img_ind])
    curr_style_effect = np.zeros((len(img_inx), style_change_effect.shape[1], style_change_effect.shape[2], style_change_effect.shape[3]))
    curr_w = np.zeros((len(img_inx), W_values.shape[1]))
    curr_style_vector_distances = np.zeros((len(img_inx), style_change_effect.shape[2], 2))
    for k, i in enumerate(img_inx):
        curr_style_effect[k, :, :] = style_change_effect[i, :, :, :]
        curr_w[k, :] = W_values[i, :]
        curr_style_vector_distances[k, :, :] = all_style_vectors_distances[i, :, :]
    style_effect_classes[img_ind] = curr_style_effect
    W_classes[img_ind] = curr_w
    style_vectors_distances_classes[img_ind] = curr_style_vector_distances
    all_style_vectors_classes[img_ind] = all_style_vectors[img_inx]
    print(f'Class {img_ind}, {len(img_inx)} images.')

Class 0, 90 images.
Class 1, 160 images.


In [14]:
def get_discriminator_results_given_dlatent(dlatent: np.ndarray,
                                            generator: tf.keras.models.Model,
                                            discriminator: tf.keras.models.Model,
                                            classifier: tf.keras.models.Model,
                                            class_index: int,
                                            sindex: int,
                                            s_style_min: float,
                                            s_style_max: float,
                                            style_direction_index: int,
                                            shift_size: float = 2,
                                            label_size: int = 2):

    network_inputs = generator.style_vector_calculator(dlatent)
    images_out = generator.g_synthesis(network_inputs, training=False)
    images_out = tf.maximum(tf.minimum(images_out, 1), -1)
    labels = tf.constant(dlatent[:, -label_size:], dtype=tf.float32)
    
    discriminator_before = discriminator([images_out, labels], training=False)
    print("output discriminator", discriminator_before)
    print("output discriminator shape", discriminator_before.shape)
  
    # I am not using the classifier output here, because it is only one.
    change_image, _ = (generate_change_image_given_dlatent(dlatent, generator, classifier,
                                                              class_index, sindex,
                                                              s_style_min, s_style_max,
                                                              style_direction_index, shift_size,
                                                              label_size))
    
    
    labels = tf.nn.softmax(classifier(change_image, training=False))
    change_image_for_disc = tf.transpose(change_image, (0, 3, 1, 2))
    discriminator_after = discriminator([change_image_for_disc, labels], training=False)
    
    return (discriminator_before, discriminator_after)

In [15]:
def discriminator_filter(style_change_effect: np.ndarray,
                         all_dlatents: np.ndarray,
                         generator: tf.keras.models.Model,
                         discriminator: tf.keras.models.Model,
                         classifier: tf.keras.models.Model,
                         sindex: int,
                         style_min: float,
                         style_max: float,
                         class_index: int,
                         num_images: int = 10,
                         label_size: int = 2,
                         change_threshold: float = 0.5,
                         shift_size: float = 2,
                         effect_threshold: float = 0.2,
                         sindex_offset: int = 0):

    for style_sign_index in range(2):
        images_idx = ((style_change_effect[:, style_sign_index, sindex, class_index]) > effect_threshold).nonzero()[0]

        images_idx = images_idx[:num_images]
        dlatents = all_dlatents[images_idx]

        for i in range(len(images_idx)):
            cur_dlatent = dlatents[i:i + 1]
            (discriminator_orig, discriminator_change) = get_discriminator_results_given_dlatent(
                                                               dlatent=cur_dlatent,
                                                               generator=generator,
                                                               discriminator=discriminator,
                                                               classifier=classifier,
                                                               class_index=class_index,
                                                               sindex=sindex + sindex_offset,
                                                               s_style_min=style_min,
                                                               s_style_max=style_max,
                                                               style_direction_index=style_sign_index,
                                                               shift_size=shift_size,
                                                               label_size=label_size)

            if np.abs(discriminator_orig - discriminator_change) > change_threshold:
                return False
    return True

In [16]:
def find_significant_styles(
    style_change_effect: np.ndarray,
    num_indices: int,
    class_index: int,
    discriminator: Optional[tf.keras.models.Model],
    generator: tf.keras.models.Model,
    classifier: tf.keras.models.Model,
    all_dlatents: np.ndarray,
    style_min: np.ndarray,
    style_max: np.ndarray,
    max_image_effect: float = 0.2,
    label_size: int = 2,
    discriminator_threshold: float = 0.2,
    sindex_offset: int = 0):

    num_images = style_change_effect.shape[0]
    style_effect_direction = np.maximum(0, style_change_effect[:, :, :, class_index].reshape((num_images, -1)))

    images_effect = np.zeros(num_images)
    all_sindices = []
    discriminator_removed = []
    while len(all_sindices) < num_indices:
        next_s = np.argmax(np.mean(style_effect_direction[images_effect < max_image_effect], axis=0))
        if discriminator is not None:
            sindex = next_s % style_change_effect.shape[2]
            if sindex == 0:
                break
            if not discriminator_filter(
                      style_change_effect=style_change_effect,
                      all_dlatents=all_dlatents,
                      generator=generator,
                      discriminator=discriminator,
                      classifier=classifier,
                      sindex=sindex,
                      style_min=style_min[sindex + sindex_offset],
                      style_max=style_max[sindex + sindex_offset],
                      class_index=class_index,
                      label_size=label_size,
                      change_threshold=discriminator_threshold,
                      sindex_offset=sindex_offset):
                style_effect_direction[:, next_s] = np.zeros(num_images)
                discriminator_removed.append(sindex)
                continue

        all_sindices.append(next_s)
        images_effect += style_effect_direction[:, next_s]
        style_effect_direction[:, next_s] = 0

    return [(x // style_change_effect.shape[2],(x % style_change_effect.shape[2]) + sindex_offset) for x in all_sindices]

In [17]:
label_size_clasifier = 2
num_indices =  10
effect_threshold = 0.2
use_discriminator = False
discriminator_model = discriminator if use_discriminator else None
s_indices_and_signs_dict = {}

for class_index in [0, 1]:
    split_ind = 1 - class_index
    all_s = style_effect_classes[split_ind]
    all_w = W_classes[split_ind]

    # Find s indicies
    s_indices_and_signs = find_significant_styles(
        style_change_effect=all_s,
        num_indices=num_indices,
        class_index=class_index,
        discriminator=discriminator_model,
        generator=generator,
        classifier=classifier,
        all_dlatents=all_w,
        style_min=style_min,
        style_max=style_max,
        max_image_effect=effect_threshold*5,
        label_size=label_size_clasifier,
        discriminator_threshold=0.2,
        sindex_offset=0)

    s_indices_and_signs_dict[class_index] = s_indices_and_signs


sindex_class_0 = [sindex for _, sindex in s_indices_and_signs_dict[0]]

all_sindex_joined_class_0 = [(1 - direction, sindex) for direction, sindex in 
                             s_indices_and_signs_dict[1] if sindex not in sindex_class_0]
all_sindex_joined_class_0 += s_indices_and_signs_dict[0]

scores = []

for direction, sindex in all_sindex_joined_class_0:
    other_direction = 1 if direction == 0 else 0
    curr_score = np.mean(style_change_effect[:, direction, sindex, 0]) + np.mean(style_change_effect[:, other_direction, sindex, 1])
    scores.append(curr_score)

s_indices_and_signs = [all_sindex_joined_class_0[i] for i in np.argsort(scores)[::-1]]
print('Directions and style indices for moving from class 1 to class 0 = ', s_indices_and_signs[:num_indices])
print('Use the other direction to move for class 0 to 1.')

Directions and style indices for moving from class 1 to class 0 =  [(1, 5300), (0, 5147), (1, 3301), (1, 3199), (1, 3921), (1, 1208), (0, 4519), (0, 5246), (0, 4172), (1, 4469)]
Use the other direction to move for class 0 to 1.


In [18]:
def get_classifier_results(generator: tf.keras.models.Model, expanded_dlatent: tf.Tensor, use_softmax: bool = False):
    image = call_synthesis(generator, expanded_dlatent)
    image = tf.transpose(image, (0, 2, 3, 1))
    results = classifier(image, training=False)
    if use_softmax:
        return tf.nn.softmax(results).numpy()[0]
    else:
        return results.numpy()[0]

In [19]:
def call_synthesis(generator: tf.keras.models.Model,
                   dlatents_in: tf.Tensor,
                   conditioning_in: Optional[tf.Tensor] = None,
                   labels_in: Optional[tf.Tensor] = None,
                   training: bool = False,
                   num_layers: int = 14,
                   dlatent_size: int = 512) -> tf.Tensor:

    if labels_in is not None:
        zero_labels = tf.zeros_like(labels_in)
        dlatents_labels = tf.tile(tf.expand_dims(labels, 1), [1, num_layers, 1])
        if dlatent_size > 0:
            dlatents_expanded = tf.concat([dlatents_in, dlatents_labels], axis=2)
        else:
            dlatents_expanded = dlatents_labels
    else:
        if dlatent_size == 0:
            raise ValueError('Dlatents are empty and no labels were provided.')
        dlatents_expanded = dlatents_in

    style_vector_blocks, style_vector_torgb = generator.style_vector_calculator(dlatents_expanded[:, 0], training=training)
    
    if conditioning_in is not None:
        network_inputs = (style_vector_blocks, style_vector_torgb, conditioning_in)
    else:
        network_inputs = (style_vector_blocks, style_vector_torgb)
        synthesis_results = generator.g_synthesis(network_inputs, training=training)

    return tf.maximum(tf.minimum(synthesis_results, 1), -1)

In [20]:
LAYER_SHAPES = []

for dense in generator.style_vector_calculator.style_dense_blocks:
    LAYER_SHAPES.append(dense.dense_bias.weights[0].shape[1])

def sindex_to_layer_idx_and_index(generator: tf.keras.models.Model,
                                  sindex: int) -> Tuple[int, int]:
    global LAYER_SHAPES
    layer_shapes_cumsum = np.concatenate([[0], np.cumsum(LAYER_SHAPES)])
    layer_idx = (layer_shapes_cumsum <= sindex).nonzero()[0][-1]
    return layer_idx, sindex - layer_shapes_cumsum[layer_idx]

In [21]:
def decode_latents(latents, generator, num_images, batch_size, image_size=(256, 256), channels=3):
    
    h, w = image_size
    
    generated_images = []
    
    for latent in tqdm.tqdm(latents):
        expanded_latent = tf.tile(tf.expand_dims(np.expand_dims(latent, 0), 1), [1, 14, 1])
        generated_image = call_synthesis(generator, expanded_latent, num_layers=14)
        generated_image = tf.transpose(generated_image, perm=[0, 2, 3, 1])
        generated_image = generated_image.cpu()
        generated_images.append(generated_image)

    generated_dataset = tf.stack(generated_images)
    generated_dataset = tf.reshape(generated_dataset, [num_images // batch_size, batch_size, h, w, channels])

    return generated_dataset

In [22]:
def encode_decode(dataset, generator, encoder, classifier, normalize_layer):
    
    generated_batches = []

    for batch in tqdm.tqdm(dataset):
        
        batch = normalize_layer(batch)
        classification = tf.nn.softmax(classifier(batch, training=False))
        batch = tf.transpose(batch, perm=[0, 3, 1, 2])

        output_encoder = encoder.predict_on_batch(batch)
        latent = tf.concat([output_encoder, classification], -1)
        expanded_latent = tf.tile(tf.expand_dims(latent, 1), [1, 14, 1])

        generated_batch = call_synthesis(generator, expanded_latent, num_layers=14)
        generated_batch = tf.transpose(generated_batch, perm=[0, 2, 3, 1])
        generated_batch = generated_batch.cpu()
        generated_batches.append(generated_batch)
        
        del batch
        del generated_batch
    
    generated_dataset = tf.stack(generated_batches).cpu()

    return generated_dataset

In [23]:
def preprocess_inception(batch, size, generated, normalize):
    
    batch = tf.image.resize(batch, size)
    if not generated:
        batch = normalize(batch)
        
    return batch

In [24]:
def create_counterfactual_dataset(k,
                                  latents,
                                  generator,
                                  classifier,
                                  s_indices_and_signs,
                                  style_min,
                                  style_max,
                                  num_images,
                                  batch_size,
                                  image_size = (256, 256),
                                  channels = 3,
                                  s_shift_size = 1):
    
    h, w = image_size
    counterfactual_images = []
    indices_and_signs = s_indices_and_signs[:k]
    print("Lenth", len(indices_and_signs))
    
    for latent in tqdm.tqdm(latents):

        latent = np.expand_dims(latent, axis=0)

        expanded_latent = tf.tile(tf.expand_dims(latent, 1), [1, num_layers, 1])
        base_prob = get_classifier_results(generator, expanded_latent)

        if np.argmax(base_prob) == 1:
            flip = False
        elif np.argmax(base_prob) == 0:
            flip = True

        classifier_results = []
        one_hots = []

        for direction, sindex in indices_and_signs:

            layer_idx, weight_idx = sindex_to_layer_idx_and_index(generator, sindex)

            layer = generator.style_vector_calculator.style_dense_blocks[layer_idx]
            layer_size = layer.dense_bias.weights[0].shape[1]

            s_vals = tf.concat(generator.style_vector_calculator(latent, training=False)[0], axis=1).numpy()[0]

            if (direction == 0 and flip == False) or (direction == 1 and flip == True):
                shift = (style_min[sindex] - s_vals[sindex]) * s_shift_size

            elif (direction == 1 and flip == False) or (direction == 0 and flip == True):
                shift = (style_max[sindex] - s_vals[sindex]) * s_shift_size

            one_hot_shift = shift * tf.expand_dims(tf.one_hot(weight_idx, layer_size), axis=0)
            one_hots.append(one_hot_shift)
            layer.dense_bias.weights[0].assign_add(one_hot_shift)
        
        counterfactual_image = call_synthesis(generator, expanded_latent, num_layers=num_layers)
        counterfactual_images.append(tf.transpose(counterfactual_image, perm=[0, 2, 3, 1]).cpu())

        for x, (_, sindex) in enumerate(indices_and_signs):
            layer_idx, weight_idx = sindex_to_layer_idx_and_index(generator, sindex)
            layer = generator.style_vector_calculator.style_dense_blocks[layer_idx]
            layer.dense_bias.weights[0].assign_add(-one_hots[x])

    counterfactual_dataset = tf.stack(counterfactual_images).cpu()
    counterfactual_dataset = tf.reshape(counterfactual_dataset, [num_images // batch_size, batch_size, h, w, channels]).cpu()
    
    return counterfactual_dataset

In [25]:
def compute_fid(model, dataset, generated_dataset, size, normalize):
    
    save_original_mu = []
    save_original_sigma = []
    save_generated_mu = []
    save_generated_sigma = []
    
    for i, batch in enumerate(tqdm.tqdm(dataset)):

        dataset_batch = preprocess_inception(batch, size, False, normalize)
        generated_dataset_batch = preprocess_inception(generated_dataset[i], size, True, normalize)
        
        original_act = model.predict(dataset_batch)
        generated_act = model.predict(generated_dataset_batch)

        original_mu, original_sigma = original_act.mean(axis=0), cov(original_act, rowvar=False)
        generated_mu, generated_sigma = generated_act.mean(axis=0), cov(generated_act, rowvar=False)
        
        save_original_mu.append(original_mu)
        save_original_sigma.append(original_sigma)
        save_generated_mu.append(generated_mu)
        save_generated_sigma.append(generated_sigma)
        
        del dataset_batch
        del generated_dataset_batch
    
    original_mu = np.mean(save_original_mu, axis=0)
    original_sigma = np.mean(save_original_sigma, axis=0)
    generated_mu = np.mean(save_generated_mu, axis=0)
    generated_sigma = np.mean(save_generated_sigma, axis=0)
    ssdiff = np.sum((original_mu - generated_mu)**2.0)
    covmean = sqrtm(original_sigma.dot(generated_sigma))

    if iscomplexobj(covmean):
        covmean = covmean.real

    fid = ssdiff + trace(original_sigma + generated_sigma - 2.0 * covmean)

    return fid

In [26]:
fid_images = "./fid_images"
original_dir = os.path.join(fid_images, "original_images")
generated_dir = os.path.join(fid_images, "generated_images")
counterfactual_dir = os.path.join(fid_images, "counterfactual_images")
fids_results_filepath = "./fid_results.csv"

if not os.path.exists(fid_images):
    os.mkdir(fid_images)
    os.mkdir(original_dir)
    os.mkdir(generated_dir)
    os.mkdir(counterfactual_dir)

In [27]:
def save_tensor_image(folder, dataset):
    for index, image in enumerate(dataset):
        image = img_to_array(image[0])
        save_img(os.path.join(folder, 'image' + str(index) + '.png'), image)

In [28]:
def fid_topk(original_dir,
             generated_dir,
             counterfactual_dir,
             k,
             latents,
             generator,
             classifier,
             inception,
             s_indices_and_signs,
             style_min,
             style_max,
             cuda_rank=0):
    
    fids = []

    fid_original_generated = calculate_fid_given_paths([original_dir, generated_dir], batch_size=50, device=cuda_rank, dims=2048)
    fids.append(fid_original_generated)
    
    for i in range(k):
        counterfactual_dataset = create_counterfactual_dataset(i+1, latents, generator,
                                                       classifier, s_indices_and_signs,
                                                       style_min, style_max,
                                                       num_images, batch_size=batch_size)
        save_tensor_image(counterfactual_dir, counterfactual_dataset)
        fid_counterfactual = calculate_fid_given_paths([original_dir, counterfactual_dir], batch_size=50, device=cuda_rank, dims=2048)
        fids.append(fid_counterfactual)
    
    return fids

In [29]:
def save_fids(fids, fids_results_filepath):
    num_fids = len(fids)
    column_names = ["Original-Generated"] + ["Original-Attribute 1"] + ["Original-Attributes 1-" + str(i) for i in range(2, num_fids)]
    data = {column_names[i]:[format(fids[i], '.2f')] for i in range(num_fids)}
    df = pd.DataFrame(data)
    df.to_csv(fids_results_filepath) 

In [30]:
batch_size = 1
num_images = latents.shape[0]
image_size = (256, 256)
channels = 3
s_shift_size = 1

original_dataset = tf.keras.utils.image_dataset_from_directory("/content/tensorflow-inception_images/Images",
                                                               labels=None,
                                                               label_mode=None,
                                                               class_names=None,
                                                               image_size=(256, 256),
                                                               batch_size=batch_size)
save_tensor_image(original_dir, original_dataset)

Found 250 files belonging to 1 classes.


In [31]:
generated_dataset = decode_latents(latents, generator, num_images, batch_size)
save_tensor_image(generated_dir, generated_dataset)

  0%|          | 0/250 [00:00<?, ?it/s]

Instructions for updating:
Use tf.identity instead.


100%|██████████| 250/250 [00:20<00:00, 12.16it/s]


In [32]:
num_attributes = 10
fids = fid_topk(original_dir,
                generated_dir,
                counterfactual_dir,
                num_attributes,
                latents, generator,
                classifier, inception,
                s_indices_and_signs,
                style_min, style_max)
save_fids(fids, fids_results_filepath)

Downloading: "https://github.com/mseitzer/pytorch-fid/releases/download/fid_weights/pt_inception-2015-12-05-6726825d.pth" to /root/.cache/torch/hub/checkpoints/pt_inception-2015-12-05-6726825d.pth


  0%|          | 0.00/91.2M [00:00<?, ?B/s]

100%|██████████| 5/5 [00:01<00:00,  3.09it/s]


78.5034961551699
Lenth 1


100%|██████████| 5/5 [00:01<00:00,  3.02it/s]


Lenth 2


100%|██████████| 5/5 [00:01<00:00,  3.07it/s]
